In [1]:
from argparse import ArgumentParser
from json import dump
from logging import basicConfig, getLogger
from os import linesep, remove
from os.path import exists
from tempfile import NamedTemporaryFile
from typing import Dict, List, Tuple

from requests import get
from sentencepiece import SentencePieceProcessor
from tqdm import trange, tqdm

In [2]:
basicConfig()

In [14]:
# class SentencePieceExtractor:
#     """
#     Extractor implementation for SentencePiece trained models.
#     https://github.com/google/sentencepiece
#     """

#     def __init__(self, model: str):
#         # Get SentencePiece
#         self.sp = SentencePieceProcessor()
#         self.sp.Load(model)

#     def extract(self) -> Tuple[Dict[str, int], List[Tuple]]:
#         sp = self.sp
#         vocab = {sp.id_to_piece(index): index for index in trange(sp.GetPieceSize())}

#         # Merges
#         merges = []
#         for piece_l in tqdm(vocab.keys(), total=sp.GetPieceSize()):
#             for piece_r in vocab.keys():
#                 merge = f"{piece_l}{piece_r}"
#                 piece_id = vocab.get(merge, None)
#                 if piece_id:
#                     merges += [(piece_l, piece_r, piece_id)]
#         merges = sorted(merges, key=lambda val: val[2])
#         merges = [(val[0], val[1]) for val in merges]

#         return vocab, merges

In [15]:
# from itertools import chain
# from multiprocessing import Pool, cpu_count

# class SentencePieceExtractor:
#     """
#     Extractor implementation for SentencePiece trained models.
#     https://github.com/google/sentencepiece
#     """

#     def __init__(self, model: str):
#         # Get SentencePiece
#         self.sp = SentencePieceProcessor()
#         self.sp.Load(model)
    
#     def extract(self) -> Tuple[Dict[str, int], List[Tuple]]:
#         sp = self.sp
#         vocab = {sp.id_to_piece(index): index for index in trange(sp.GetPieceSize())} 

#         results = []
#         with Pool(cpu_count()) as pool:
#             results = pool.starmap(self.extract_merges, [(key, vocab,) for key in vocab.keys()])

#         # Flatten and filter empty lists
#         merges = list(chain.from_iterable(filter(None, results)))

#         merges.sort(key=lambda val: val[2])
#         merges = [(val[0], val[1]) for val in merges]
        
#         return vocab, merges

#     """
#     Multiprocessing for merges.
#     """
#     @staticmethod
#     def extract_merges(piece_l, vocab):
#         merges = []
#         for piece_r in vocab.keys():
#             merge = f"{piece_l}{piece_r}"
#             piece_id = vocab.get(merge, None)
#             if piece_id:
#                 merges += [(piece_l, piece_r, piece_id)]
        
#         return merges

In [16]:
import sys
sys.path.insert(0, "/projects/data/aamod/Libraries/tokenizers/bindings/python/scripts")
from sentencepiece_extractor import SentencePieceExtractor

ModuleNotFoundError: No module named 'sentencepiece_extractor'

In [3]:
sp = SentencePieceProcessor()
sp.Load("/DATA/rohit/NLP_2025/bpe_mixed.model")

True

In [4]:
spe = SentencePieceExtractor("/DATA/rohit/NLP_2025/unigram_sanskrit.model")

NameError: name 'SentencePieceExtractor' is not defined

In [ ]:
v, m = spe.extract()

NameError: name 'spe' is not defined

In [ ]:
v1, m1 = spe.extract()

NameError: name 'spe' is not defined

In [ ]:
v1 == v

True

In [ ]:
m1 == m

True

In [ ]:
v

In [5]:
vocab = {sp.id_to_piece(index): index for index in trange(sp.GetPieceSize())}

100%|██████████| 3200/3200 [00:00<00:00, 393704.29it/s]


In [6]:
from multiprocessing import Pool, cpu_count

In [7]:
def process_merges(piece_l, vocab):    
    merges = []
    for piece_r in vocab.keys():
        merge = f"{piece_l}{piece_r}"
        piece_id = vocab.get(merge, None)
        if piece_id:
            merges += [(piece_l, piece_r, piece_id)]
    
    return merges

In [21]:
## Think about sorting later, aim should be correctness

In [8]:
results = []
with Pool(cpu_count()) as pool:
    results = pool.starmap(process_merges, [(k, vocab,) for k in vocab.keys()])

In [9]:
merges = []
for i in results:
    if i != []:
        merges += i

In [10]:
merges = sorted(merges, key=lambda val: val[2])
merges = [(val[0], val[1]) for val in merges]

In [11]:
import json

In [12]:
with open("vocab_fast.json", "w") as f:
    json.dump(vocab, f)

In [13]:
type(merges)

list

In [14]:
merges[0]

('▁', 'क')

In [15]:
with open("merges_fast.txt", "w") as f:
    for item in merges:
        f.write(f"{item[0]} {item[1]}\n")

In [16]:
from tokenizers import SentencePieceBPETokenizer

In [17]:
tokenizer = SentencePieceBPETokenizer()

In [18]:
test = tokenizer.from_file("vocab_fast.json", "merges_fast.txt")

In [19]:
test

Tokenizer(vocabulary_size=3200, model=SentencePieceBPE, unk_token=<unk>, replacement=▁, add_prefix_space=True, dropout=None)

In [20]:
test.save_model("/DATA/rohit/NLP_2025/mt_bpe")

['/DATA/rohit/NLP_2025/mehak/vocab.json',
 '/DATA/rohit/NLP_2025/mehak/merges.txt']

In [21]:
test.save("/DATA/rohit/NLP_2025/mt_bpe/bpe/tokenizer.json")

In [ ]:
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast(tokenizer_file="Final/tokenizer.json")

In [ ]:
tokenizer.save_pretrained("Paka_Final")

('Paka_Final/tokenizer_config.json',
 'Paka_Final/special_tokens_map.json',
 'Paka_Final/tokenizer.json')

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Paka_Final", use_fast=True)

In [ ]:
tokenizer.encode("How are tyou sff <BOS>ffds dsa<EOS>1!)")

[13576,
 6047,
 11980,
 5304,
 5315,
 5522,
 124378,
 3,
 5318,
 83741,
 5348,
 11795,
 4,
 124378,
 124400,
 64684]